## Target number of units

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
buildings_sql = '''
SELECT  SUM(COALESCE(residential_units,0)) AS residential_units
FROM urbansim.urbansim.building
'''

In [ ]:
du_df =  pd.read_sql(buildings_sql, mssql_engine)

In [ ]:
du = int(du_df.values)

In [ ]:
print "Existing units:", du

In [ ]:
household_sql = '''
  SELECT sum(hh) AS hh
  FROM isam.demographic_output.summary
  WHERE sim_id = 1004 and yr = 2050
  '''

In [ ]:
hh_df =  pd.read_sql(household_sql, mssql_engine)

In [ ]:
hh = int(hh_df.values)

In [ ]:
print "Number of households in 2050:", hh

In [ ]:
target_for_region = hh - du

In [ ]:
print target_for_region